# Machine Learning-Based Transcriptomic Biomarker Discovery in Huntington's Disease

**Project:** Discovery of Key Biomarkers for Huntington's Disease Using Meta-Analysis and Machine Learning
**Data Source:** GSE64810 (Pre-processed & Filtered)
**Models:** Random Forest (Feature Selection) & Support Vector Machine (Classification)

---

## 1. Setup and Data Loading
Import necessary libraries and upload the `HD_ML_Ready_Data.csv` file.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score, roc_curve

# Setup plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [ ]:
from google.colab import files

print("Please upload 'HD_ML_Ready_Data.csv'")
uploaded = files.upload()

# Check if file is uploaded
filename = list(uploaded.keys())[0]
print(f"Loaded file: {filename}")

data = pd.read_csv(filename)
data.head()

## 2. Data Preprocessing
Encoding the target variable and splitting the data into training and testing sets.

In [ ]:
# Separate Features (Genes) and Target
X = data.drop('Target_Class', axis=1)
y = data['Target_Class']

# Encode Target (Control -> 0, HD -> 1)
le = LabelEncoder()
y_encoded = le.fit_transform(y)
class_names = le.classes_
print(f"Class Mapping: {dict(zip(class_names, le.transform(class_names)))}")

# Split Data (70% Train, 30% Test) - Stratified to maintain class balance
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.3, stratify=y_encoded, random_state=42
)

# Scale data for SVM (SVM requires feature scaling)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"Training samples: {X_train.shape[0]}")
print(f"Testing samples: {X_test.shape[0]}")
print(f"Number of Features (Genes): {X_train.shape[1]}")

## 3. Model Training
Training Random Forest (for feature importance) and SVM (for high accuracy).

In [ ]:
# --- Random Forest ---
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# --- SVM (Linear Kernel) ---
# probability=True is needed for ROC-AUC
svm_model = SVC(kernel='linear', probability=True, random_state=42)
svm_model.fit(X_train_scaled, y_train)

print("Models trained successfully.")

## 4. Evaluation and Visualization
Generating predictions, confusion matrices, and ROC curves.

In [ ]:
# Predictions
y_pred_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1]

y_pred_svm = svm_model.predict(X_test_scaled)
y_prob_svm = svm_model.predict_proba(X_test_scaled)[:, 1]

# --- Metrics Function ---
def print_metrics(name, y_true, y_pred, y_prob):
    acc = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, y_prob)
    print(f"\n--- {name} Results ---")
    print(f"Accuracy: {acc:.4f}")
    print(f"ROC-AUC: {auc:.4f}")
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    return confusion_matrix(y_true, y_pred)

cm_rf = print_metrics("Random Forest", y_test, y_pred_rf, y_prob_rf)
cm_svm = print_metrics("SVM", y_test, y_pred_svm, y_prob_svm)

In [ ]:
# --- Plot Confusion Matrices ---
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', ax=axes[0], 
            xticklabels=class_names, yticklabels=class_names)
axes[0].set_title('Random Forest Confusion Matrix')
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')

sns.heatmap(cm_svm, annot=True, fmt='d', cmap='Greens', ax=axes[1], 
            xticklabels=class_names, yticklabels=class_names)
axes[1].set_title('SVM Confusion Matrix')
axes[1].set_ylabel('Actual')
axes[1].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

# --- Plot ROC Curves ---
fpr_rf, tpr_rf, _ = roc_curve(y_test, y_prob_rf)
fpr_svm, tpr_svm, _ = roc_curve(y_test, y_prob_svm)

plt.figure(figsize=(8, 6))
plt.plot(fpr_rf, tpr_rf, label=f'Random Forest (AUC = {roc_auc_score(y_test, y_prob_rf):.2f})', lw=2)
plt.plot(fpr_svm, tpr_svm, label=f'SVM (AUC = {roc_auc_score(y_test, y_prob_svm):.2f})', lw=2)
plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves')
plt.legend(loc='lower right')
plt.show()

## 5. Feature Importance (Biomarker Identification)
Visualizing the top 20 genes that the Random Forest model used to make decisions.

In [ ]:
importances = rf_model.feature_importances_
indices = np.argsort(importances)[::-1][:20] # Top 20 features

top_genes = X.columns[indices]
top_importances = importances[indices]

plt.figure(figsize=(10, 8))
sns.barplot(x=top_importances, y=top_genes, palette='viridis')
plt.title('Top 20 Candidate Biomarkers (Random Forest Importance)')
plt.xlabel('Importance Score')
plt.ylabel('Gene ID')
plt.show()

print("Top 10 Genes:")
for gene, score in zip(top_genes[:10], top_importances[:10]):
    print(f"{gene}: {score:.4f}")

## 6. Save and Download Models
We will save the trained models, the scaler (crucial for SVM), and the list of feature names (crucial for ensuring input consistency). We'll zip them together for a single download.

In [ ]:
# Create a directory for artifacts
!mkdir -p hd_models

# 1. Save Random Forest Model
joblib.dump(rf_model, 'hd_models/rf_model.pkl')

# 2. Save SVM Model
joblib.dump(svm_model, 'hd_models/svm_model.pkl')

# 3. Save Scaler (Required for SVM inference)
joblib.dump(scaler, 'hd_models/scaler.pkl')

# 4. Save Feature Names (To ensure future input order matches training)
feature_names = list(X.columns)
joblib.dump(feature_names, 'hd_models/feature_names.pkl')

# 5. Save Label Encoder (To decode 0/1 back to Control/HD)
joblib.dump(le, 'hd_models/label_encoder.pkl')

print("Artifacts saved locally in 'hd_models/' folder.")

# Zip the folder
!zip -r hd_models.zip hd_models

# Download the zip file
files.download('hd_models.zip')

## 7. Inference Demo
This section demonstrates how to load the saved models and run predictions on new data.

In [ ]:
def predict_hd_status(new_data_df, model_type='svm'):
    """
    Runs inference on new data.
    Args:
        new_data_df (pd.DataFrame): Dataframe with Gene IDs as columns.
        model_type (str): 'svm' or 'rf'
    Returns:
        Prediction string (HD or Control)
    """
    # Load artifacts
    try:
        rf_loaded = joblib.load('hd_models/rf_model.pkl')
        svm_loaded = joblib.load('hd_models/svm_model.pkl')
        scaler_loaded = joblib.load('hd_models/scaler.pkl')
        features_loaded = joblib.load('hd_models/feature_names.pkl')
        le_loaded = joblib.load('hd_models/label_encoder.pkl')
    except FileNotFoundError:
        return "Error: Model files not found. Run the training steps first."
    
    # Validate Features
    # Ensure new_data has the exact same columns in same order
    try:
        new_data_df = new_data_df[features_loaded]
    except KeyError:
        missing = set(features_loaded) - set(new_data_df.columns)
        return f"Error: Input data is missing genes: {list(missing)[:5]}..."

    # Prediction logic
    if model_type == 'svm':
        # SVM requires scaling
        data_scaled = scaler_loaded.transform(new_data_df)
        pred_idx = svm_loaded.predict(data_scaled)
        prob = svm_loaded.predict_proba(data_scaled)[:, 1]
    else:
        # RF does not require scaling
        pred_idx = rf_loaded.predict(new_data_df)
        prob = rf_loaded.predict_proba(new_data_df)[:, 1]
        
    # Decode prediction
    prediction = le_loaded.inverse_transform(pred_idx)
    
    results = pd.DataFrame({
        'Prediction': prediction,
        'Probability_HD': prob
    })
    return results

# --- Test Inference using a sample from our test set ---
# (Simulating 'new' data by taking 2 rows from X_test)
sample_input = X_test.iloc[:2].copy()
print("\nRunning Inference on Sample Data:")
print(predict_hd_status(sample_input, model_type='svm'))